<a href="https://colab.research.google.com/github/JacopoMangiavacchi/NER_Training/blob/main/WNUT17/train_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Get data

In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.6MB/s 
     |████████████████████████████████| 890kB 24.4MB/s 
     |████████████████████████████████| 2.9MB 40.9MB/s 
     |████████████████████████████████| 1.1MB 37.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d6f0575adb15b587d0b576436d60d80eeb11d3f82ad0d0989441c8272911322d
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
!wget http://noisy-text.github.io/2017/files/wnut17train.conll

--2020-11-04 22:47:37--  http://noisy-text.github.io/2017/files/wnut17train.conll
Resolving noisy-text.github.io (noisy-text.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to noisy-text.github.io (noisy-text.github.io)|185.199.108.153|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 493781 (482K) [application/octet-stream]
Saving to: ‘wnut17train.conll’

wnut17train.conll   100%[===================>] 482.21K  --.-KB/s    in 0.04s   

2020-11-04 22:47:38 (11.3 MB/s) - ‘wnut17train.conll’ saved [493781/493781]



In [3]:
from pathlib import Path
import re

def read_wnut(file_path):
    file_path = Path(file_path)

    raw_text = file_path.read_text().strip()
    raw_docs = re.split(r'\n\t?\n', raw_text)
    token_docs = []
    tag_docs = []
    for doc in raw_docs:
        tokens = []
        tags = []
        for line in doc.split('\n'):
            token, tag = line.split('\t')
            tokens.append(token)
            tags.append(tag)
        token_docs.append(tokens)
        tag_docs.append(tags)

    return token_docs, tag_docs

texts, tags = read_wnut('wnut17train.conll')

In [4]:
list(set([str(t) for line in tags for t in line]))

['O',
 'I-corporation',
 'B-group',
 'I-creative-work',
 'I-person',
 'I-group',
 'B-person',
 'B-creative-work',
 'B-corporation',
 'B-product',
 'I-location',
 'I-product',
 'B-location']

In [5]:
print(texts[0][10:17], tags[0][10:17], sep='\n')

['for', 'two', 'weeks', '.', 'Empire', 'State', 'Building']
['O', 'O', 'O', 'O', 'B-location', 'I-location', 'I-location']


In [6]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_tags, val_tags = train_test_split(texts, tags, test_size=.2)

In [7]:
len(train_texts), len(val_texts)

(2715, 679)

In [8]:
unique_tags = set(tag for doc in tags for tag in doc)
tag2id = {tag: id for id, tag in enumerate(unique_tags)}
id2tag = {id: tag for tag, id in tag2id.items()}

In [9]:
id2tag

{0: 'O',
 1: 'I-corporation',
 2: 'B-group',
 3: 'I-creative-work',
 4: 'I-person',
 5: 'I-group',
 6: 'B-person',
 7: 'B-creative-work',
 8: 'B-corporation',
 9: 'B-product',
 10: 'I-location',
 11: 'I-product',
 12: 'B-location'}

# Use pre-trained DistilBert Tokenizer

In [10]:
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-cased')
train_encodings = tokenizer(train_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)
val_encodings = tokenizer(val_texts, is_split_into_words=True, return_offsets_mapping=True, padding=True, truncation=True)

In [11]:
train_encodings.keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping'])

In [12]:
len(train_encodings['input_ids']), len(val_encodings['input_ids'])

(2715, 679)

In [13]:
len(train_encodings['input_ids'][0])

90

# Format labels for sub token splitting

In [14]:
import numpy as np

def encode_tags(tags, encodings):
    labels = [[tag2id[tag] for tag in doc] for doc in tags]
    encoded_labels = []
    for doc_labels, doc_offset in zip(labels, encodings.offset_mapping):
        # create an empty array of -100
        doc_enc_labels = np.ones(len(doc_offset),dtype=int) * -100
        arr_offset = np.array(doc_offset)

        # set labels whose first offset position is 0 and the second is not 0
        doc_enc_labels[(arr_offset[:,0] == 0) & (arr_offset[:,1] != 0)] = doc_labels
        encoded_labels.append(doc_enc_labels.tolist())

    return encoded_labels

train_labels = encode_tags(train_tags, train_encodings)
val_labels = encode_tags(val_tags, val_encodings)

In [15]:
len(train_labels), len(val_labels)

(2715, 679)

In [16]:
len(train_labels[0])

90

# Create Dataset loader

In [17]:
import torch

class WNUTDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_encodings.pop("offset_mapping") # we don't want to pass this to the model
val_encodings.pop("offset_mapping")
train_dataset = WNUTDataset(train_encodings, train_labels)
val_dataset = WNUTDataset(val_encodings, val_labels)

# Reuse DistilBert model

In [18]:
from transformers import DistilBertForTokenClassification
model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this 

# Option 1: Fine Tune DistilBert using HuggingFace Training class

In [19]:
# from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

# training_args = TrainingArguments(
#     output_dir='./results',          # output directory
#     num_train_epochs=3,              # total number of training epochs
#     per_device_train_batch_size=16,  # batch size per device during training
#     per_device_eval_batch_size=64,   # batch size for evaluation
#     warmup_steps=500,                # number of warmup steps for learning rate scheduler
#     weight_decay=0.01,               # strength of weight decay
#     logging_dir='./logs',            # directory for storing logs
#     logging_steps=10,
# )

# trainer = Trainer(
#     model=model,                         # the instantiated 🤗 Transformers model to be trained
#     args=training_args,                  # training arguments, defined above
#     train_dataset=train_dataset,         # training dataset
#     eval_dataset=val_dataset             # evaluation dataset
# )

# trainer.train()

# Option 2: Fine Tune DistilBert using PyTorch

In [20]:
from torch.utils.data import DataLoader
from transformers import DistilBertForSequenceClassification, AdamW

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Reload model as already trained in option 1
# model = DistilBertForTokenClassification.from_pretrained('distilbert-base-cased', num_labels=len(unique_tags))
model.to(device)
model.train()

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

optim = AdamW(model.parameters(), lr=5e-5)

for epoch in range(10):
    epoch_loss = 0.0
    num_batch = 0
    for i, batch in enumerate(train_loader, 0):
        optim.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs[0]
        loss.backward()
        optim.step()
        #print(f"Epoch {epoch} Batch {i} : Loss {loss.data}")
        epoch_loss += loss.item()
        num_batch += 1
    print(f"Epoch {epoch} : Loss {epoch_loss / num_batch}")



Epoch 0 : Loss 0.30093758531353054
Epoch 1 : Loss 0.15261841179693447
Epoch 2 : Loss 0.07888876687434009
Epoch 3 : Loss 0.04305682856014327
Epoch 4 : Loss 0.020932647355777375
Epoch 5 : Loss 0.012041078556007157
Epoch 6 : Loss 0.0071772040917387455
Epoch 7 : Loss 0.00607396115665324
Epoch 8 : Loss 0.004868980471784358
Epoch 9 : Loss 0.00698583277330994


# Validate Model

In [21]:
testloader = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [22]:
predictions = []
actual = []
with torch.no_grad():
   for i, batch in enumerate(testloader, 0):
     input_ids = batch['input_ids'].to(device)
     attention_mask = batch['attention_mask'].to(device)
     outputs = model(input_ids, attention_mask)
     predictions.append(outputs[0].cpu())
     actual.append(batch['labels'])
predictions = np.concatenate(predictions)
actual = np.concatenate(actual)

In [23]:
predicted_tokens = np.argmax(predictions, axis=2)

In [24]:
predicted_tags = []

for p_tokens, a_tokens in zip(predicted_tokens, actual):
  a_mask = a_tokens!=-100
  word_tokens = p_tokens[a_mask]
  iob_tags = [id2tag[x] for x in word_tokens]
  predicted_tags.append(iob_tags)

In [25]:
!pip install seqeval

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=2339deb403b8dfe67e96c05fe2f663979f17e7641d3f4eb7cd66e3242189b096
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [26]:
from seqeval.metrics import f1_score

In [27]:
f1_score(val_tags, predicted_tags)

0.5940054495912807